In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19192207/tmp32q4zdgt
  JVM stdout: /state/partition1/job-19192207/tmp32q4zdgt/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19192207/tmp32q4zdgt/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_mk7516_jgp3e1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.543 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(1596)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(435759, 55)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |█████████████████████
14:37:15.77: GLM_1_AutoML_1_20220507_141758 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: 54

██████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_1_20220507_141758

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.15527525614321736
RMSE: 0.3940498142915656

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.18789363063584152
RMSE: 0.43346698909587283

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.746586,0.001388,0.746122,0.745294,0.745421,0.748362,0.747728
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.253414,0.001388,0.253878,0.254706,0.254579,0.251638,0.252272
3,err_count,22085.600000,183.155120,22045.000000,22395.000000,22085.000000,21972.000000,21931.000000
4,logloss,0.577103,0.002727,0.577423,0.579615,0.578878,0.576987,0.572612
5,max_per_class_error,0.762054,0.014593,0.783764,0.767057,0.760553,0.753653,0.745242
6,mean_per_class_accuracy,0.620227,0.006241,0.611261,0.620933,0.624520,0.617283,0.627140
7,mean_per_class_error,0.379773,0.006241,0.388739,0.379067,0.375480,0.382717,0.372860
8,mse,0.187893,0.000736,0.187912,0.188626,0.188569,0.187450,0.186906
9,null_deviance,210045.890000,1317.521200,208974.120000,212212.450000,209289.520000,210362.190000,209391.200000


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20220507_141758,0.379666,0.577108,0.433467,0.187894
StackedEnsemble_BestOfFamily_2_AutoML_1_20220507_141758,0.381498,0.60685,0.446816,0.199644
StackedEnsemble_AllModels_1_AutoML_1_20220507_141758,0.384017,0.590213,0.44039,0.193944
GBM_1_AutoML_1_20220507_141758,0.395499,0.579929,0.43379,0.188174
XGBoost_1_AutoML_1_20220507_141758,0.411763,0.609917,0.446116,0.199019
XGBoost_2_AutoML_1_20220507_141758,0.436148,0.652975,0.466361,0.217493
DRF_1_AutoML_1_20220507_141758,0.469489,0.794016,0.478107,0.228587
GBM_4_AutoML_1_20220507_141758,0.477265,1.11671,0.66016,0.435811
GBM_3_AutoML_1_20220507_141758,0.495174,1.07816,0.646076,0.417414
GBM_2_AutoML_1_20220507_141758,0.501293,1.03926,0.631277,0.39851
